# Progetto Intermedio - Parte 1 

# Indice

- [Inizializzazione del notebook](#inizial)
  - [Installazioni](#instal)
  - [Import](#import)
  - [Utils](#utils)
- [Importazione ed elaborazione dei dati](#dati)
  - [Lettura dati](#lett_dati)
- [Sistema di raccomandazione](#sis)
  - [Bag-of-words lemmatizing](#bow_lemm)
  - [KNN BOW - Lemmatizing](#knn_bow_lemm)
  - [Bag-of-words stemming](#bow_stem)
  - [KNN BOW - Stemming](#knn_bow_stem)


# Inizializzazione del notebook <a class="anchor"  id="inizial"></a>

## Installazioni <a class="anchor"  id="instal"></a>

In [1]:
!pip install datasets

In [2]:
!pip install nltk
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

## Import <a class="anchor"  id="import"></a>

In [4]:
from datasets import load_dataset

from surprise.model_selection import train_test_split

from bokeh.io import output_notebook
output_notebook()

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Loading BokehJS ...

In [5]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [6]:
stop_words.update([',','.','!','?',';','-','...','[',']','{','}','(',')','&',':', '--','<','>','^','#','\\','/','\'','\"','br', '<br />', '\'s', '&#34', '34', 'n\'t', '\`', "''", '``'])

## Utils <a class="anchor"  id="utils"></a>

In [7]:
def text_processing(sentences, method="lemmatize"):
    tokenized_sentences = []
    for i, text in enumerate(sentences) :
        word_token = word_tokenize(text)
        tokens = []
        for word in word_token:
            if word.casefold() not in stop_words:
                if method == "lemmatize":
                    tokens.append(lemmatizer.lemmatize(word))
                else:
                    tokens.append(stemmer.stem(word))
        tokenized_sentences.append(tokens)
    return tokenized_sentences

In [8]:
def cut_dataset(reviews, min_n_reviews_user, min_n_reviews_item):
    user_review_count = reviews.groupby("user_id").count()[["parent_asin"]].reset_index()
    print("numero di utenti: ", len(user_review_count[user_review_count["parent_asin"] > 0]))
    print("numero di utenti con un numero di recensioni > ", min_n_reviews_user, " : ", len(user_review_count[user_review_count["parent_asin"] > min_n_reviews_user]))
    users_id = user_review_count[user_review_count["parent_asin"] > min_n_reviews_user]["user_id"].unique().tolist()

    item_review_count = reviews.groupby("parent_asin").count()[["user_id"]].reset_index()
    print("numero di item: ", len(item_review_count[item_review_count["user_id"] > 0]))
    print("numero di item con un numero di recensioni > ", min_n_reviews_item, " : ", len(item_review_count[item_review_count["user_id"] > min_n_reviews_item]))
    items_id = item_review_count[item_review_count["user_id"] > min_n_reviews_item]["parent_asin"].tolist()

    reviews = reviews[reviews["user_id"].isin(users_id)].reset_index(drop=True)
    reviews = reviews[reviews["parent_asin"].isin(items_id)].reset_index(drop=True)

    # users_id e items_id vengono "risettati" in quanto può essere che un utente abbia dato due recensioni (e quindi venga incluso nella prima lista)
    # ma che le due recensioni appartengano a prodotti che hanno solo una recensione e quindi vengono eliminate le reviews -> quindi gli utenti (o gli item)
    # in questione non hanno più reviews (e il numero diminuisce)

    return reviews, reviews["user_id"].unique(), reviews["parent_asin"].unique()

In [9]:
def add_title(df):
    df["text"] = df["title"]+ " " +df["text"]
    return df

In [10]:
def create_bag_of_words(tokenized_sentences):# create the vocabulary
    vocab = set()

    # create the bag-of-words model
    bow_model = []

    for tokens in tokenized_sentences:
        # create a dictionary to store the word counts
        word_counts = {}

        # update the vocabulary
        vocab.update(tokens)

        # count the occurrences of each word
        for word in tokens:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1

        # add the word counts to the bag-of-words model
        bow_model.append(word_counts)
    return bow_model, vocab

In [11]:
def create_bow_data(vocab, descriptions_data, items_info):
    data = pd.DataFrame(0, index=range(len(items_info)), columns=list(vocab))
    for i in range(len(items_info)):
        if i % 1000 == 0: 
            print(i)
        data.loc[i, descriptions_data[i].keys()] = descriptions_data[i].values()
        
    data["parent_asin"] = items_info["parent_asin"]
    return data

In [12]:
def split_and_test(data, rating_col_name, test_size = 0.20, random_state = 0,n_neighbors = 30, metric = "cosine"):
    #faccio la stessa cosa per ogni user
    mse_users = []
    i = 0
    for user_id in users_id:
        i = i + 1
        if i % 100 == 0:
            print(i)
            
        # item valutati dall'utente
        user_ratings = reviews[reviews['user_id'] == user_id][["parent_asin", "rating", "user_id"]]
        rated_items = data[data['parent_asin'].isin(user_ratings['parent_asin'])]

        
        # creo il dataset
        dataset = pd.merge(rated_items, user_ratings, on="parent_asin")
        dataset = dataset.drop(columns=["parent_asin", "user_id"])
        if len(rated_items) == 0:
            continue
            
        # Split train/test
        try:
            X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=[rating_col_name]),
                                                        pd.Series(dataset[rating_col_name]),
                                                        test_size=test_size,
                                                        train_size= 1 - test_size,
                                                        random_state=random_state)
            # Train k-NN
            neigh_reg = KNeighborsRegressor(n_neighbors=min(n_neighbors, len(X_train)),
                                            metric=metric, n_jobs = -1)
            neigh_reg.fit(X_train, y_train)
            # Test k-NN
            y_pred = neigh_reg.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            mse_users.append(mse)
        

        except:
            continue
    
    print(f"Average MSE over users: {np.mean(mse_users):.2f}")
    print(f"Average RMSE over users: {np.sqrt(np.mean(mse_users)):.2f}")

# Importazione ed elaborazione dei dati <a class="anchor"  id="dati"></a>

## Lettura dati <a class="anchor"  id="lett_dati"></a>

In [13]:
reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Software", trust_remote_code=True)
reviews = reviews["full"].to_pandas()
items_info = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Software", split="full", trust_remote_code=True)

Generating full split: 0 examples [00:00, ? examples/s]

Generating full split: 0 examples [00:00, ? examples/s]

In [14]:
reviews, users_id, items_id = cut_dataset(reviews, 30, 60)

numero di utenti:  2589466
numero di utenti con un numero di recensioni >  30  :  3027
numero di item:  89246
numero di item con un numero di recensioni >  60  :  7007


In [15]:
print(len(users_id))
print(len(items_id))

3025
6059


In [16]:
#togliamo da items_info tutte quelle righe (items) che non hanno parent_asin in items_id
items_info = items_info.filter(lambda row: row["parent_asin"] in items_id)

Filter:   0%|          | 0/89251 [00:00<?, ? examples/s]

In [17]:
len(items_info)

6059

In [18]:
descriptions = items_info["description"]

In [19]:
#se un oggetto ha più descrizioni la compattiamo in un'unica stringa
for i in range(len(descriptions)):
    description = descriptions[i]
    if len(description) >= 1:
        flattened_description = ''
        for j in range(len(description)):
            flattened_description = flattened_description + description[j] + ' '
        descriptions[i] = flattened_description
    elif len(description) == 0:
        descriptions[i] = ''

In [20]:
print(descriptions[:5])

["Just Escape, whether it's a medieval castle or an abandoned space station you'll need to solve puzzles and find clues to unlock the door and Just Escape. ", 'Tax Software that helps you\xa0get your taxes done right and your maximum refund Tax Software that helps you\xa0get your taxes done right and your maximum refund We’ll search more than 350 deductions and credits (1040, Schedule A) to help make tax preparation easy We’ll search more than 350 deductions and credits (1040, Schedule A) to help make tax preparation easy Get expert answers to your questions by phone (fees may apply) by Intuit Get expert answers to your questions by phone (fees may apply) by Intuit Income Tax Software that accurately deduct mortgage interest and property taxes Income Tax Software that accurately deduct mortgage interest and property taxes Includes one TurboTax State product download (State efile not included) Includes one TurboTax State product download (State efile not included) ', 'Frozen Characters 

In [21]:
for i in range(len(descriptions)):
    descriptions[i] = items_info[i]["title"] + " " +  descriptions[i]

In [22]:
descriptions[:5]

["Just Escape Just Escape, whether it's a medieval castle or an abandoned space station you'll need to solve puzzles and find clues to unlock the door and Just Escape. ",
 'TurboTax Deluxe 2014 Fed + State + Fed Efile Tax Software - Win [Download] OLD VERSION Tax Software that helps you\xa0get your taxes done right and your maximum refund Tax Software that helps you\xa0get your taxes done right and your maximum refund We’ll search more than 350 deductions and credits (1040, Schedule A) to help make tax preparation easy We’ll search more than 350 deductions and credits (1040, Schedule A) to help make tax preparation easy Get expert answers to your questions by phone (fees may apply) by Intuit Get expert answers to your questions by phone (fees may apply) by Intuit Income Tax Software that accurately deduct mortgage interest and property taxes Income Tax Software that accurately deduct mortgage interest and property taxes Includes one TurboTax State product download (State efile not incl

# Sistema di raccomandazione <a class="anchor"  id="sis"></a>

## Bag-of-words lemmatizing <a class="anchor"  id="bow_lemm"></a>

In [23]:
descriptions_tokens= text_processing(descriptions, "lemmatize")

In [24]:
bow_lemmatized_descriptions, bow_vocab_lemmatized = create_bag_of_words(descriptions_tokens)

## KNN BOW - Lemmatizing <a class="anchor"  id="knn_bow_lemm"></a>

In [25]:
bow_lemmatized_data = create_bow_data(bow_vocab_lemmatized, bow_lemmatized_descriptions, items_info)

0
1000
2000
3000
4000
5000
6000


In [26]:
bow_lemmatized_data

,www.twitter.com/bigfishgamesTALK,Cook'n,'Race,TurtlesThat,fast.With,shortest,metal,pre-wedding,game.Mahjong,PPM,...,Lollipops,fulfilling,collections*,impede,children.You,game-Wolf,Single-Lens,Wildly,PIP,parent_asin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B00K7BMELK
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B00NG7JVSQ
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B00JVOJ5T8
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B06XXH983G
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B008K6IB5C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6054,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B002ABOYXG
6055,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B005ZKC4FO
6056,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B077HV61JX
6057,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,B07H4PVKFL


In [27]:
split_and_test(bow_lemmatized_data, "rating_y", test_size = 0.20, random_state = 0, n_neighbors = 10, metric = "cosine")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
Average MSE over users: 1.27
Average RMSE over users: 1.13


## Bag-of-words stemming <a class="anchor"  id="bow_stem"></a>


In [28]:
descriptions_tokens= text_processing(descriptions,"stem")

In [29]:
bow_stemmed_descriptions, bow_vocab_stemmed = create_bag_of_words(descriptions_tokens)

## KNN BOW - Stemming <a class="anchor"  id="knn_bow_stem"></a>


In [30]:
bow_stemmed_data = create_bow_data(bow_vocab_stemmed, bow_stemmed_descriptions, items_info)

0
1000
2000
3000
4000
5000
6000


In [31]:
split_and_test(bow_stemmed_data, "rating", test_size = 0.20, random_state = 0, n_neighbors = 10, metric = "cosine")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
Average MSE over users: 1.27
Average RMSE over users: 1.13
